In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.prompts.chat import AIMessagePromptTemplate
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
# few shots

examples = [
    {"challenge": "A German driver is about to approach your bus stop, tell him in German: I am alighting", 
     "student_response": "Ich steige aus", 
     "tutor_feedback": "Bravo! Seems you're really gonna be fluent soon"},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Sie sind frei!", 
     "tutor_feedback": "Good try but the article used stands for You. Try to use the article for We"},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Wir sind frei!", 
     "tutor_feedback": "Great work! But there is just a little bit left."},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Wir sind jetzt frei!", 
     "tutor_feedback": "Excellent! Keep on practicing your German skills"},
]

def generate_few_shot_messages(examples: list):
    all_msgs = []
    for example in examples:
        all_msgs.append(HumanMessagePromptTemplate.from_template(f"""
        Challenge: {example["challenge"]}
        Student's Response: {example["student_response"]}
        """))
        all_msgs.append(AIMessagePromptTemplate.from_template(f"Tutor's Feedback: {example['tutor_feedback']}"))

    return all_msgs

few_shot_msgs = generate_few_shot_messages(examples)

In [15]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
        You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.
        Given a challenge, and your student's response, you give helpful feedback to guide the student to the right solution.
        You give all your feedback in English.
        Avoid giving any translations of the answer from German to English.
        If the student fails, try to guide them not translating the answer.
        """                               
    )] + few_shot_msgs + [
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
    ])

In [16]:
llm = ChatOpenAI(temperature=0)

conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationBufferMemory(return_messages=True),
    verbose=True)

In [17]:
input_prompt = """Challenge: How would you tell your German girl: I have never eaten anything that sweet before
                  Student's Response: Haette Ich gewusst dass es du waere, haette ich dich besser gekusst.
                """
response = conversation.predict(input=input_prompt)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: 
        You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.
        Given a challenge, and your student's response, you give helpful feedback to guide the student to the right solution.
        You give all your feedback in English.
        Avoid giving any translations of the answer from German to English.
        If the student fails, try to guide them not translating the answer.
        
Human: 
        Challenge: A German driver is about to approach your bus stop, tell him in German: I am alighting
        Student's Response: Ich steige aus
        
AI: Tutor's Feedback: Bravo! Seems you're really gonna be fluent soon
Human: 
        Challenge: How would your German friend tell you, We are free now!
        Student's Response: Sie sind frei!
        
AI: Tutor's Feedback: Good try but the article used stands for You.